# 06.Transfer Laerning with TensorFlow Part 3 : Scaling up( Food vision mini)

## What we're going to cover

We're oging to go throguh the follow with TensorFlow:
 - Downloading and preparing 10% of the Food101 data(10% of trainig data)
 - Training a feature extraction transfer learnin model on 10% of the Fooe101 training data
 - Fine-tuning our feature extraction model
 - Saving and loaded our traned model
 - Evaluating the performance of our Food Vision model trained on 10% of the training data
    - Finding our model's most wrong predicions
- making predicions with our Food Vision model on cumstom image of food

In [ ]:
# Are we using a GPU?
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")

Notebook last run (end-to-end): 2025-06-02 07:09:58.205157


## Create helper functions

In [ ]:
# Get helper functions file
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2025-06-02 07:09:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-06-02 07:09:58 (78.2 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## 101 Food Classes: Working with less data

## Downloading and preprocessing the data

In [ ]:
# Download data from Google Storage (already preformatted)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test"


--2025-06-02 07:09:58--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G   189MB/s    in 15s     

2025-06-02 07:10:13 (102 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



NameError: name 'unzip_data' is not defined

In [ ]:
# How many images/classes are there?
walk_through_dir("101_food_classes_10_percent")

In [ ]:
# Setup data inputs
import tensorflow as tf
IMG_SIAE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir, label_mode = "categorical",
                                                                                image_size = IMG_SIZE)
test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir, label_mode = "categorical",
                                                              image_size=IMG_SIZE,
                                                              shuffle=False)


## Train a big dog model with transfer learning on 10% ot 101 food classes

In [ ]:
# Create checkpoint callback to save model for later use
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.Modelcheckpoint(checkpoint_path,
                                                         save_weights_only = True, # save only the model weights
                                                         monitor = "val_accuracy", # save the model weigghts which score the best validation accuracy
                                                         save_best_only = True) # only keep the best model weights on file (delete the rest)

In [ ]:
# import the required modules for model creation
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

data_augmentaion = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2).

], name = "data_augmentation")

In [ ]:
# Setup base model and freeae its layers (this will extract features)
base_model =tf.keras.applications.efficientnet.EfficientNetB0(include_top=Fasle)
base_model.trainable = False

# Setup model architerue with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name = "input_layer" )# shape of input image
X = data_augementaion(inputs) # augment images ( only happens during training)
x = base_model(x, trainig = False) # put the base model in inference mode so we can useit to extract feature without update the weights
x = layers.GlobalAveragePooling2D(name = "global_average_pooling")(x) # poolt the outputs ot the base model
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name = "output_layer")(x) # same number of outputs as classes
model = tf.keras.Model(inputs, outputs)


In [ ]:
# Get a summary of our model
model.summary()

In [ ]:
# compile
model.compile(loss = "cateforical_crossentropy",
              optimizers = tf.keras.optimizers.Adam().
              metrics = ["accuracy"])

# fit
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs = 5, # fit for epochs to keep experiments quick
                                           validation_data = test_data,
                                           validation_steps = int(0.15 * len(test_data)),
                                           callbacks = [checkpoint_callback])

In [ ]:
# Evaluate model
results_feature_extraction_model = model.evaluate(test_data)
results_feature_extraction_model

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

## Fine-tunning

In [ ]:
# Unfreeze all ot the layer in the base model
base_model.trainable = True

# Refreeze every layer except for the last 5
for layer in base_model.layer[:-5]:
    layer.trainable = False


In [ ]:
# Recompile model with lower learning rate
model.compile(loss='categoricla_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-4), # 10x lwer learning rate than default
              metrics =['accuracy'])

In [ ]:
# What layers in the model are trainable?
for layer in model.layers:
    print(layer.name, layer.trainable)

In [ ]:
# check which layers are trainable
for layer_number, layer in enumerate(base_model.layers):
    print(layer_number, layer.name, layer.trainable)

In [ ]:
# Fine-Tune for 5 more epochs
fine_tune_epochs = 10 # model hase already done 5 epochs, this is the total number of epochs we're after (5+5=10)

history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs = fine_tune_epochs,
                                                     valdiation_data = test_data,
                                                     validation_steps = int(0.15 * len(test_data)), # validation on 15% of the test data
                                                     intial_epoch = history_all_classes_10_percent.epoch[-1])# start from previous last epoch

In [ ]:
# Evaluate fine-tunned model on the whole test dataset
results_all_classes_10_percent_fine_tune = model.evaluate(test_data)
results_all_classes_10_percent_fine_tune

##  Saving our trained model


In [ ]:
## save model to drive so it can be used later
model.save("drive/My Drvie/tensorflow_course/101_food_class_10_percent_saved_big_dog_model")

## Evaluate the performance of the big dog model across all different classes

In [ ]:
import tensorflow as tf

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip
saved_model_path = "06_101_food_class_10_percent_saved_big_dog_model.zip"
unzip_data(saved_model_path)

model = tf.keras.models.load_model(saved_model_path.split(".")[0]) # don't include ".zip" in loaded model path

In [ ]:
# Check to see if loaded model is a trained model
loaded_loss, loaded_accuracy = model.evaluate(test_data)
loaded_loss, loaded_accuracy

In [ ]:
# Make predictions with model
pred_probs = model.predict(test_data, verbose = 1) # set verbosity to see how long it will take

In [ ]:
# How many predicions are there?
len(pred_probs)

In [ ]:
# What's the shape of our predictions?
pred_probs.shape

In [ ]:
# How do they look?
pred_probs[:10]

In [ ]:
# we get one predition probability per class
print(f"Number of predictions probabilities for sample 0: {len(pred_probs[0])}")
print(f"What prediction probability sample 0 looks like: \n {pred_probs[0]}")
print(f"The class with the highest predicted probability by the model for sample 0: {pred_probs[0].argmax()}")

In [ ]:
# Get the class predicions of each label
pred_classes = pred_probs.srgmax(axis = 1)

# How do thdy look?
pred_classes[:10]

In [ ]:
y_labels = []
for images, labels in test_data.unbatch(): # unbatch the test data and get images and labels
    y_labels.append(labels.numpy().argmax()) # append the index which has the largest value (labels are one-hot)
y_labels[:10] # check what they look like (unshuffled)

In [ ]:
# How many labels are there ? (should be the same as how many predictions probabilites we have)
len(y_labels)

## Evaluating our models predictions

In [ ]:
# Get accuracy score by comparing predicted classes to ground truth labels
from sklearn.metrics import accuracy_score
sklearn_accuracy = accrucay_scroe(y_labels, pred_classes)
sklearn_accuracy

In [ ]:
# Does the evaluate method compare to the Scikit-learn measured accracy?
import numpy as np
print(f"Close?{np.isclose(loaded_accuracy, sklearn_accuracy)} | Difference : {loaded_accuracy - sklearn_accuracy}")

In [ ]:
from helper_functions import make_confusion_matrix

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10,10), test_size =15, norm=False, savefig=False):

    # Create the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_norm  = cm.astype("float") / cm.sum(axis = 1)[:, np.newaxis] # normalize it
    n_classes = cm.shape[0] # find the number of classes we're dealing with

    # plot the figure and mske it pretty
    fig, ax= plt.subplots(figsize = figsize)
    cax = ax.matshow(cm, cmap = plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
    fig.colorbas(cax)

    # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])

    ax.set(title = "Confusion Matrix",
           xlabel = "Predicted label",
           ylabel = "True label",
           xticks = np.arnage(n_classes), # create enogh axis slots for each class
           yticks = np.arange(n_classes),
           xticklabels = labels, # axes will labeled with class name (if they exist) or ints
           yticklabels = labels)

    # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("botton")
    ax.xaxis.tick_bottom()

    ### Added : Rotate xticks for readability & increate font siae (requured due to such a large confusion matrix)
    plt.xticks(rotation=70, fontsize = test_size)
    plt.yticks(fontsize = test_size)

    # Set the threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.

     # Plot the test on each cell
     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(j, i, f"{cm[i, j]*100 :.1f}%",
                     horizontalaignment="center",
                     color = "white" if cm[i,j] > threshold else "black",
                     size = test_size)
        else:
            plt.text(j, i, f"{cm[i, j]}",
                    horizontalalignment = "center",
                    color = "white" if cm[i,j] > threshold else "black",
                    size = text_size)

    # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")

In [ ]:
# Get the class names
class_name = test_data.class_names
class_name[:10]

In [ ]:
# Plot a confusion matrix with all 25250 prdicions, ground truth labels and 101 calsses
make_confusion_matrix(y_true = y_labels,
                      y_pred=pred_classes,
                      classes = class_names,
                      figsize = (100,100),
                      text_size = 20,
                      norm = False,
                      savefig = True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels, pred_classes))

In [ ]:
# Get a dictionary ot the classification report
classification_report_dict = classification_report(y_labels, pred_classes, output_dict = True)
classification_report_dict

In [ ]:
# Create empty dictionary
class_f1_scores = {}
# Loop through classification reprot items
for k, v in classification_reprot_dict.items():
    if k == "accuracy": # stop once we get to accuracy key
        break
    else:
        # Append class names and f1-scored to new dictionary
        class_f1_scores[class_names[int(k)]] = v["f1-score"]
class_f1_scores

In [ ]:
# Turn f1-score into dataframe for visualization
import pandas as pd
f1_score = pd.DataFrame({"class_name" : list(class_f1_scores.keys()),
                         "f1-score": list(class_f1_scores.values())}).sort_values("f1-score", ascending = False)
                         f1_score.head()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplot(figsize = (12,25))
scores = ax.barh(range(len(f1_scores)), f1_scores["f1_score"].values)
ax.set_yticks(range(len(f1_scores)))
ax.set_ytickslabels(list(f1_scores["class_name"]))
ax.set_xlabel("f1-score")
ax.set_title("F1-scores for 10 Different Classes")
ax.invert_yaxis(); # reverse the order

def autolabel(rects) :

    for rect in rects:
       width = rect.get_width()
       ax.test(1.03*width, rect.get_Y() + rect.get_height()/1.5,
               f"width:.2f",
               ha='center',
               va = "bottom")
autolabel(scores)

## Visualizing predictions on thest images

In [ ]:
def load_and_prep_image(filename, img_shape = 224, scale=True):

    # Read in the image
    img = tf.io.read_file(filename)

    # Decode it into a tensor
    img = tf.io.decode_image(img)

    # Resize the image
    img = tf.image.resize(img, [img_shape, img_shape])
    if scale:
        # Rescale the image( get all values between 1 and 1)
        return img/255.
    else:
        return img


In [ ]:
# Make preds on a series of random iamges
import os
import random

plt.figure(figsize=(17, 10))
for i in range(3):
    # Choose a random image from a random class
    class_name = random.choice(class_names)
    filename = random.choice(os.listdir(test_dir + "/" + class_name))
    filepath = test_dir + class_name + "/" filename

    # Load the image and make predictions
    img = load_and_prep_image(filepath, scale = False) # don't scale images for Efficient predictions
    pred_prob = model.predict(tf.expand_dims(img, axis = 0)) # model accepts tensors of shape [ None, 224, 224, 3]
    pred_class = class_name[pred_prob, argmax()] # find the predicted class

    # Plot the iamge(s)
    plt.subplot(1, 2, i+1)
    plt.imshow(img/255.)
    if class_name == pred_class: # change the color of the test based on whether prediction is right or wrong
        title_color= "g"
    else:
        title_color ="r"
    plt.title(f"acrual : {class_name}, pred  : {pred_class}, prop : {pred_prob.max():.2f}", c=title_color)
    plt.axis(False);

In [ ]:
# 1. Get the filenames of all of our test data

filepaths  = []
for filepath in test_data.list_files("101_food_classes_10_percent/test/*/*.jpg",
                                     shuffle = False):
    filepaths.append(filepath.numpy())
filepaths[:10]


In [ ]:
# 2. Create a dataframe out of current predicion data for analysis
import pandas as pd
pred_df = pd.DataFrame({"img_path": filepaths,
                        "y_true": y_labels,
                        "y_pred" : pred_classes,
                        "pred_conf" : pred_probs.mas(axs = 1), # get the maximum prediction probability value
                        "y_true_classname":[class_names[i] for i in y_labels],
                        "y_pred_classname" : [class_names[i] for i in pred_classes]})
pred_df.head()

In [ ]:
# 3. Is the prediction correct?
pred_df["pred_correct"] = pred_dr["y_true"] ==  pred_df["y_pred"]
pred_df.head()

In [ ]:
# 4. Get the top 100 wrong examples
top_100_wrong  = pred_df[pred_df["pred_correct"] == False].sort_values("pred_conf", ascending = False)[:100]
top_100_wrong.head(20)

In [ ]:
# 5. Visualize some of the most wrong examples
image_to_view = 9
start_index = 10 # change the start index to view more
plt.figure(figsize = (15, 10))
for i, fow in enumerate(top_100_wrong[start_index: start_index + images_to_view].itertuples()):
    plt.subplot(3, 3, i+1)
    img = load_and_prep_image(row[1], scale = True)
    -, -,-,-, pred_prob, y_true, y_pred, - = row # only interested in a fes parameters of each row
    plt.imshow(img)
    plt.title(f"actual : {y_true}, pred : {y_pred} \n prob : {pred_prob:.2f}")
    plt.axis(Flase)

## Test out the big dog model on test images as well as custom images of food


In [ ]:
# Download some custom images from Google Storage
# Note: you can upload your own custom images to Google Colab using the "upload" button in the Files tab
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/custom_food_images.zip

unzip_data("custom_food_images.zip")

In [ ]:
# Get custom food image filepaths
custom_food_images = ["custom_food_images/" + img_path for img_path in os.listdir("custom_food_images")]
custome_food_images

In [ ]:
# Make predicitions on custom food images
for img in custom_food_images:
    img = load_and_prep_image(img, scale = Flase) # Load in target image and turn it into
    pred_prob = model.predict(tf.exand_dims(img, axis = 0l))
    pred_calss = class_name[pred_prob.argmax()] # find the predited class label
    # Plot the image with appropriate annotations
    plt.figure()
    plt.imshow(img/255.)
    plt.title(f"pred : {pred_class}, prob : {pred_prob.max():.2f}")
    plt.axis(False)